In [19]:
from bridge_specs import box
span = box['span']

In [6]:
class cross_section:
    def __init__(self, distance, span_length):
        self.distance = distance
        self.span_length = span_length
    class cables:
        def __init__(self, number):
            self.number = number

In [25]:
end_cables_pos = [(0,1), (0,0), (1,0), (1,1)]
mid_cables_pos = [(0,0.25), (0,0), (1,0), (1,0.25)]
all_cable_pos = zip(end_cables_pos, mid_cables_pos)
for i in all_cable_pos:
    print(i)

((0, 1), (0, 0.25))
((0, 0), (0, 0))
((1, 0), (1, 0))
((1, 1), (1, 0.25))


### Cable profile

In [36]:
def cable_profile(end_cable_pos, mid_cable_pos, span_length, division=8):
    cable_profile = []
    x = [(j)/division*span_length for j in range(division+1)]
    for i in x:
        cable_pos_x = []
        for j in range(len(end_cable_pos)):
            h = end_cable_pos[j][1]-mid_cable_pos[j][1]
            a = end_cable_pos[j][1]
            y = a + 4*h*i/span_length - 4*h*i**2/span_length**2
            cable_pos_x.append((end_cable_pos[j][0], y))
        cable_profile.append(cable_pos_x)
    return cable_profile

In [37]:
cable_profile(end_cables_pos, mid_cables_pos, 50)

[[(0, 1.0), (0, 0.0), (1, 0.0), (1, 1.0)],
 [(0, 1.328125), (0, 0.0), (1, 0.0), (1, 1.328125)],
 [(0, 1.5625), (0, 0.0), (1, 0.0), (1, 1.5625)],
 [(0, 1.703125), (0, 0.0), (1, 0.0), (1, 1.703125)],
 [(0, 1.75), (0, 0.0), (1, 0.0), (1, 1.75)],
 [(0, 1.703125), (0, 0.0), (1, 0.0), (1, 1.703125)],
 [(0, 1.5625), (0, 0.0), (1, 0.0), (1, 1.5625)],
 [(0, 1.328125), (0, 0.0), (1, 0.0), (1, 1.328125)],
 [(0, 1.0), (0, 0.0), (1, 0.0), (1, 1.0)]]

### Moment of inertia including cable

In [6]:
def MOI(cable_positions, cable_area, cable_moi, section_moi, centroid):
    cable_moi_sum = 0
    for i in cable_positions:
        cable_moi_sum = cable_area*(i[1]-centroid[1])**2
    moi = len(cable_positions)*cable_moi + section_moi + cable_moi_sum

### Eccentricity of cables

In [11]:
def eccentricity(cable_positions, cable_area, centroid):
    cable_area_sum = len(cable_positions)*cable_area
    cable_areaxpos_sum = 0
    for i in cable_positions:
        cable_areaxpos_sum+=cable_area*(centroid[1]-i[1])
    return cable_areaxpos_sum/cable_area_sum

eccentricity(cables_pos, 1000, (0.50, 1))

0.5

### Stress